In [1]:
import numpy as np
import pandas as pd
import math, gc

import sympy as sym
from IPython.display import display, Latex

## Chapter 4 The Term Structure Of Interest

### 4.1 The Yield Curve

- Yield plotted against the maturity of a bond of similar nature.

### 4.2 The Term Structure

- Spot Rate ($s_t$): Interest rate on money held from present $t = 0$ to $t$
  - e.g. spot rates are seen in:
    - Annual Compound Interest (a)
    - Compound Interest of m periods per year (b)
    - Continuous Compound (c)

### 4.3 Forward Rates

- Forward Rate: future yield of bond given two future time periods

In [25]:
def impliedForwardRate_a(t_1, s_1, t_2, s_2):
    """
    t_1: starting period
    s_1: spot rate at period t_1
    t_2: end period
    s_2: spot rate at period t_2
    f_12: implied forward rate (under annual compound)
    assert t_1 < t_2
    """
    f_12 = ((((1 + s_2) ** t_2) / (1 + s_1 ** t_1)) ** (1 / (t_2 - t_1))) - 1
    return f_12

In [26]:
def impliedForwardRate_b(t_1, s_1, t_2, s_2):
    """
    t_1: starting period
    s_1: spot rate at period t_1
    t_2: end period
    s_2: spot rate at period t_2
    f_12: implied forward rate (under compound of m periods per year)
    assert t_1 < t_2
    """
    f_12 = (m * ((((1 + (s_2 / m)) ** t_2) / ((1 + (s_1 / m)) ** t_1)) ** (1 / (t_2 - t_1)))) - m
    return f_12

In [27]:
def impliedForwardRate_c(t_1, s_1, t_2, s_2):
    """
    t_1: starting period
    s_1: spot rate at period t_1
    t_2: end period
    s_2: spot rate at period t_2
    f_12: implied forward rate (under continuous compound)
    assert t_1 < t_2
    """
    f_12 = ((s_2 * t_2) - (s_1 * t_1)) / (t_2 - t_1)
    return f_12

### 4.4 Explanations for Term Structure

1. Expectations Theory
2. Liquidity Preference
3. Market Segmentation

### 4.5 Expectations Dynamics

- Spot Rate Forecasts (via "Expectations Dynamics"):
  - Assuming:
    - Current Spot-Rate-Curve: $(s_1, s_2, ..., s_n)$
    - Spot-Rate-Curve of the following year: $(s'_1, s'_2, ..., s'_n)$
  - The current forward rate can be thought of as expectations of what interest rates will be the following year. This, thus, allows measuremnt of interest rate from next year until $(t_2 - 1)$ years ahead.
  - i.e. $f_{1,t_2}$ becomes $s'_{t_2 -1}$.

In [28]:
def spotRateForecast(s_1, t_2, s_2):
    """
    s_1: spot rate at period t_1
    t_2: end period
    s_2: spot rate at period t_2
    """
    f_1_t2 = impliedForwardRate_a(t_1=1, s_1=s_1, t_2=t_2, s_2=s_2)
    s_prime_t2_minus_1 = f_1_t2
    return s_prime_t2_minus_1

- e.g. (4.5): "simple spot rate forecast given the spot rate curve:"
  - $(6.00, 6.45, 6.8, 7.10, 7.36, 7.56, 7.77)$

In [29]:
table = pd.DataFrame({'current': [6.00, 6.45, 6.8, 7.10, 7.36, 7.56, 7.77]})
table.index += 1
table = table.transpose()

In [30]:
table.add_prefix('s_')

,s_1,s_2,s_3,s_4,s_5,s_6,s_7
current,6.0,6.45,6.8,7.1,7.36,7.56,7.77


In [31]:
table.transpose().assign(forecast=table.transpose().apply(lambda s_t2: np.round(spotRateForecast(s_1=6.00, t_2=s_t2.index, s_2=s_t2), 1)).shift(-1)).transpose().add_prefix('s_')

,s_1,s_2,s_3,s_4,s_5,s_6,s_7
current,6.0,6.45,6.8,7.1,7.36,7.56,7.77
forecast,6.9,7.20,7.5,7.7,7.90,8.10,NaN


- Discount Factors (between two future periods) in terms of Forward Rate

In [33]:
def discountFactor_t1_t2(t_1, s_1, t_2, s_2):
    """
    t_1: starting period
    s_1: spot rate at period t_1
    t_2: end period
    s_2: spot rate at period t_2
    d_t1_t2: discount factor between two future periods in terms of forward rate f_t1_t2
    assert t_1 < t_2
    """
    d_t1_t2 = (1 / (1 + impliedForwardRate_a(t_1=t_1, s_1=s_1, t_2=t_2, s_2=s_2))) ** (t_2 - t_1)
    return d_t1_t2

- Short Rates
- Invariance Theorem:
  - Assuming that interest varies under expectations dynamics:
  - Under annual compound, the total amount of money invested in the interest rate market for $n$ years will increase by $(1 + s_n)^n$ (as long as all the money is invested), regardless of the investment and/or reinvestment strategies.

### 4.6 Running Present Value

In [34]:
def runningPresentValue(cf_list, df_list):
    """
    cf_list: cashflow in list
    df_list: discount factor (computed from term structure in table 4.2)
    pv_list: running present value
    assert len(cf_list) == len(df_list)
    """
    pv_list = []
    for k in range(len(cf_list))[::-1]:
        try:
            pv_k = cf_list[k] + df_list[k] * cf_list[k+1]
        except IndexError:
            pv_k = cf_list[k]
        pv_k = np.round(pv_k, 2)
        pv_list.append(pv_k)
    
    return pv_list[::-1]

- e.g. (4.7):
  - Cash Flow: $(20, 25, 30, 35, 40, 30, 20, 10)$
  - Discount Rates: $(0.943, 0.935, 0.93, 0.926, 0.923, 0.921, 0.917, )$

In [35]:
cf_list = [20, 25, 30, 35, 40, 30, 20, 10]
df_list = [0.943, 0.935, 0.93, 0.926, 0.923, 0.921, 0.917, np.nan]

In [36]:
pd.DataFrame({'Cash Flow': cf_list, 'Discount Factors': df_list}).transpose().add_prefix('Year: ')

,Year: 0,Year: 1,Year: 2,Year: 3,Year: 4,Year: 5,Year: 6,Year: 7
Cash Flow,20.000,25.000,30.00,35.000,40.000,30.000,20.000,10.0
Discount Factors,0.943,0.935,0.93,0.926,0.923,0.921,0.917,NaN


In [37]:
runningPresentValue(cf_list, df_list)

[43.58, 53.05, 62.55, 72.04, 67.69, 48.42, 29.17, 10]

### 4.7 Floating-Rate Bonds

- Floating-Rate Bonds have fixed face value and maturity, but varying (updating / resetting) coupon payment based on the latest short-term interest.
- Thus, the exact value of future coupon payments is uncertain until the reset: seemingly difficult to assess the value of this bond.

- Theorem (4.1): The value of floating-rate bonds is equal to their par value at the time of repricing.
  - Proof: Running Present Value.

### 4.8 Duration

- Fisher-Weil Duration: duration as a weighted average of the present values at the time of cash flow.
  - Assuming the following (is given):
    - Cash Flow: $(x_{t_0}, x_{t_1}, x_{t_2}, ..., x_{t_n})$
    - Spot-Rate-Curve $(s_t, t_0 <= t <= t_n)$
    - Continuous Compound

In [38]:
math.e

2.718281828459045

In [39]:
def fisherWeilDuration(cf_list, spot_rate_curve_list):
    """
    cf_list: cashflow in list
    spot_rate_curve_list: spot rates (corresponding to periods of cf_list) in list
    D_FW: Fisher-Weil Duration
    """
    PV = 0
    n = len(cf_list)
    for i in range(n):
        PV += cf_list[i] * (math.e ** (-1 * spot_rate_curve_list[i] * i))
    
    D_FW = 0
    for i in range(n):
        D_FW += i * cf_list[i] * (math.e ** (-1 * spot_rate_curve_list[i] * i))
    D_FW *= 1 / PV
    
    return D_FW

- Quasi-Modified Duration: relative price sensitivity of a parallel shift of the spot rate curve (under discretely defined periods)

In [40]:
def quasiModifiedDuration(m, cf_list, spot_rate_curve_list):
    """
    cf_list: cashflow in list
    spot_rate_curve_list: spot rates (corresponding to periods of cf_list) in list
    D_Q: Quasi-modified Duration
    """
    PV = 0
    n = len(cf_list)
    for i in range(n):
        PV += cf_list[i] * (math.e ** (-1 * spot_rate_curve_list[i] * i))
    
    D_Q = 0
    for k in range(n):
        k += 1
        D_Q += (k / m) * cf_list[k - 1] * ((1 + (spot_rate_curve_list[k - 1] / m)) ** (-1 * (k + 1)))
    D_Q *= (1 / PV)
    
    return D_Q